In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import numpy as np

In [2]:
# Conectar al servidor MySQL
conn = mysql.connector.connect(
    user='root', password='Andrea90+', host='localhost'
)
cursor = conn.cursor()

In [3]:
#borrar la base de datos
cursor.execute("DROP DATABASE IF EXISTS `siniestros_viales`")

In [4]:
# Crear base de datos
db_name = 'siniestros_viales'
try:
    cursor.execute(f"CREATE DATABASE {db_name}")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_DB_CREATE_EXISTS:
        print(f"Database {db_name} already exists.")
    else:
        print(err.msg)

# Cerrar la conexión inicial
# cursor.close()
# conn.close()



In [5]:
#Podemos cerrar la conexión inicial y abrir una nueva con la base de datos creada, para comenzar a trabajar con ella:

cursor.close()
conn.close()


In [6]:
# Conectar a la nueva base de datos:
conn = mysql.connector.connect(
    user='root', password='Andrea90+', host='localhost', database=db_name
)
cursor = conn.cursor()

In [7]:
# Función para insertar datos
def insert_data(cursor, table_name, data):
    placeholders = ', '.join(['%s'] * len(data.columns))
    columns = ', '.join(data.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    cursor.executemany(sql, data.values.tolist())


### A medida que se crean las tablas,insertamos las tablas creadas en MySQL:

pip install pandas mysql-connector-python


Paso 1: Leer los archivos CSV con pandas

Paso 2: Insertar datos en MySQL

In [8]:
# Leer archivos CSV
dim_tiempo = pd.read_csv('Datasets_para_MySQL/dim_tiempo.csv')
dim_ubicacion = pd.read_csv('Datasets_para_MySQL/dim_ubicacion.csv',dtype={'POS_X': object, 'POS_Y': object})
dim_caracteristicas_accidente = pd.read_csv('Datasets_para_MySQL/dim_caracteristicas_accidente.csv')
dim_victima = pd.read_csv('Datasets_para_MySQL/dim_victima.csv')
hechos_accidentes = pd.read_csv('Datasets_para_MySQL/hechos_accidentes.csv')
hechos_victimas = pd.read_csv('Datasets_para_MySQL/hechos_victimas.csv')


# Leer el archivo CSV en un nuevo DataFrame, especificando el tipo de datos



In [9]:
#ver los datos del archivo para crear la tabla
dim_tiempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   FECHA    696 non-null    object 
 1   AAAA     696 non-null    int64  
 2   MM       696 non-null    int64  
 3   DD       696 non-null    int64  
 4   HORA     696 non-null    object 
 5   HH       696 non-null    float64
 6   FechaID  696 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 38.2+ KB


In [10]:
# Crear tablas
cursor.execute('''
CREATE TABLE DimTiempo (
    FechaID INT PRIMARY KEY,
    FECHA DATE,
    AAAA INT,
    MM INT,
    DD INT,
    HORA TIME,
    HH INT
);
''')

In [11]:
# Insertar datos en cada tabla
insert_data(cursor, 'DimTiempo', dim_tiempo)

In [12]:
# Confirmar la transacción para ir testeando en MySQL: 
conn.commit()

In [13]:
cursor.execute('''
CREATE TABLE DimUbicacion (
    UbicacionID INT PRIMARY KEY,
    LUGAR_DEL_HECHO VARCHAR(255),
    TIPO_DE_CALLE VARCHAR(255),
    CALLE VARCHAR(255),
    DIRECCION_NORMALIZADA VARCHAR(255),
    COMUNA VARCHAR(255),
    POS_X DECIMAL(10,8) NULL,
    POS_Y DECIMAL(10,8) NULL
);
''')

In [14]:
#Despues de mucho probar, hay una transformacion que recien se puede hacer ahora, ya que cada vez que pasamos los datos limpios a un nuevo archivo, gralmente xlsx(que guarda mas fielmente los datos que un csv), al abrirse luego para seguir trabajando pandas abre los None como NaN, y genera un conflicto en MySQL. EL error del que estamos hablando está en las columnas 'POS_X' y 'POST_Y' que tienen en algunas celdas el valor '.', el cual que no es importado ya que el tipo de dato que se quiere guardar en la BBDD es un DECIMAL , por lo que se debe guardar un None, equivalente a NULL en SQL, pero la transformacion se debe hacer con numpy primero, ya que si se hace con pandas, se guardara un string 'None' en vez de un None, y no se podra guardar en la base de datos, por lo que se debe hacer con numpy primero, y luego con pandas, para que se guarde correctamente en la base de datos.
# Paso 1: Reemplazar '.' con np.nan
dim_ubicacion['POS_X'].replace('.', np.nan, inplace=True)
dim_ubicacion['POS_Y'].replace('.', np.nan, inplace=True)



/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/196496130.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dim_ubicacion['POS_X'].replace('.', np.nan, inplace=True)
/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/196496130.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [15]:
# Paso 2: Reemplazar np.nan con None

dim_ubicacion['POS_X'].replace(np.nan, None, inplace=True)
dim_ubicacion['POS_Y'].replace(np.nan, None, inplace=True)

/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/119378974.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dim_ubicacion['POS_X'].replace(np.nan, None, inplace=True)
/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/119378974.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [16]:
insert_data(cursor, 'DimUbicacion', dim_ubicacion)

In [17]:
# Confirmar la transacción para ir testeando en MySQL: 
conn.commit()

In [18]:
dim_caracteristicas_accidente.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662 entries, 0 to 661
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TIPO_DE_CALLE              662 non-null    object
 1   CALLE                      662 non-null    object
 2   DIRECCION_NORMALIZADA      662 non-null    object
 3   VICTIMA                    662 non-null    object
 4   CaracteristicaAccidenteID  662 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 26.0+ KB


In [20]:
cursor.execute('''
CREATE TABLE DimCaracteristicasAccidente (
    CaracteristicaAccidenteID INT PRIMARY KEY,
    TIPO_DE_CALLE VARCHAR(255),
    CALLE VARCHAR(255),
    DIRECCION_NORMALIZADA VARCHAR(255),
    VICTIMA VARCHAR(255)
);
''')

In [21]:
insert_data(cursor, 'DimCaracteristicasAccidente', dim_caracteristicas_accidente)

In [22]:
# Confirmar la transacción para ir testeando en MySQL: 
conn.commit()

In [23]:
dim_victima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ROL                  698 non-null    object 
 1   SEXO                 698 non-null    object 
 2   EDAD                 653 non-null    float64
 3   FECHA_FALLECIMIENTO  698 non-null    object 
 4   VictimaID            698 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 27.4+ KB


In [24]:
cursor.execute('''
CREATE TABLE DimVictima (
    VictimaID INT PRIMARY KEY,
    ROL VARCHAR(255),
    SEXO VARCHAR(255),
    EDAD INT,
    FECHA_FALLECIMIENTO DATE
);
''')

In [25]:
#dado el mismo caso de error que en la tabla DimUbicacion, vamos a ver en que columnas estan los nan:
dim_victima.isnull().sum()



ROL                     0
SEXO                    0
EDAD                   45
FECHA_FALLECIMIENTO     0
VictimaID               0
dtype: int64

In [26]:
# Una vez detectado que el problema, en este caso los nan estan en la colomna edad, Paso a 1: Reemplazar '.' con np.nan
dim_victima['EDAD'].replace('NaN', np.nan, inplace=True)


/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/1492659485.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dim_victima['EDAD'].replace('NaN', np.nan, inplace=True)


In [27]:
# Paso 2: Reemplazar np.nan con None

dim_victima['EDAD'].replace(np.nan, None, inplace=True)


/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/2494070672.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dim_victima['EDAD'].replace(np.nan, None, inplace=True)


In [28]:
#Ahora nos aparece el problema de 'SD' en la columna FECHA_FALLECIMIENTO, que no es un valor valido para una fecha, por lo que se debe reemplazar por None:
dim_victima['FECHA_FALLECIMIENTO'].replace('SD', None, inplace=True)

/var/folders/bf/f2bv9fls2wl6xxl27gbh6lnc0000gn/T/ipykernel_26171/1435407903.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dim_victima['FECHA_FALLECIMIENTO'].replace('SD', None, inplace=True)


In [29]:
#Aparecio un nuevo error de formato de fecha en la fila 436(que en realidad es el 435, ya que la fila 1 es el encabezado), vamos a convertirla al formato correcto
dim_victima.loc[435, 'FECHA_FALLECIMIENTO'] = pd.to_datetime(dim_victima.loc[435, 'FECHA_FALLECIMIENTO'], format='%d/%m/%Y').strftime('%Y-%m-%d')



In [30]:
dim_victima.loc[435, 'FECHA_FALLECIMIENTO']

'2019-03-26'

In [31]:
insert_data(cursor, 'DimVictima', dim_victima)

In [32]:
# Confirmar la transacción para ir testeando en MySQL: 
conn.commit()

In [33]:
hechos_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261114 entries, 0 to 261113
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   HechoID                    261114 non-null  int64 
 1   ID                         261114 non-null  object
 2   N_VICTIMAS                 261114 non-null  int64 
 3   FechaID                    261114 non-null  int64 
 4   UbicacionID                261114 non-null  int64 
 5   CaracteristicaAccidenteID  261114 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 12.0+ MB


In [34]:
#Antes de agregar esta tabla, dejamos comentado que tuvimos que suplementar una columna 'HechoID', no pudimos elegir ID porque por las caracteristicas de los merges se repetia y causaba el efecto 'duplicacion'... 

cursor.execute('''
CREATE TABLE HechosAccidentes (
    HechoID INT PRIMARY KEY,
    ID VARCHAR(20),
    N_VICTIMAS INT,
    FechaID INT,
    UbicacionID INT,
    CaracteristicaAccidenteID INT,
    FOREIGN KEY (FechaID) REFERENCES DimTiempo(FechaID),
    FOREIGN KEY (UbicacionID) REFERENCES DimUbicacion(UbicacionID),
    FOREIGN KEY (CaracteristicaAccidenteID) REFERENCES DimCaracteristicasAccidente(CaracteristicaAccidenteID)
    );
''')

In [35]:
insert_data(cursor, 'HechosAccidentes', hechos_accidentes)

In [36]:
# Confirmar la transacción para ir testeando en MySQL: 
conn.commit()

In [37]:
hechos_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7863 entries, 0 to 7862
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   HechoVictimaID  7863 non-null   int64 
 1   ID_HECHO        7863 non-null   object
 2   FechaID         7863 non-null   int64 
 3   VictimaID       7863 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 245.8+ KB


In [38]:
#Debido a el 'clasico' error de los NaN, vamos a reemplazarlos por '':

hechos_victimas.fillna('', inplace=True)

In [39]:

cursor.execute('''
CREATE TABLE HechosVictimas (
    HechoVictimaID INT PRIMARY KEY,
    ID_HECHO VARCHAR(20),
    FechaID INT,
    VictimaID INT,
    FOREIGN KEY (FechaID) REFERENCES DimTiempo(FechaID),
    FOREIGN KEY (VictimaID) REFERENCES DimVictima(VictimaID)
    );
''')

In [40]:
insert_data(cursor, 'HechosVictimas', hechos_victimas)

In [41]:
# Confirmar la transacción para ir testeando en MySQL: 
conn.commit()

In [42]:
# Cerrar la conexión
cursor.close()
conn.close()